# Testing Your Code

## Why Testing Matters

When you write code to perform statistical analyses or implement numerical methods, how do you know it works correctly? You might run a few examples by hand and check the output, but this approach has serious limitations. What happens when you change the code later? What if an edge case slips through? What if a colleague modifies a function you depend on?

Testing provides a systematic way to verify that code behaves as expected. For scientific computing and statistical software, testing is especially critical:

* *Correctness:* Statistical methods often have subtle requirements. An off-by-one error in a variance calculation or an incorrect normalization constant can produce plausible but wrong results.

* *Reproducibility:* Tests document expected behavior. When you or others return to the code months later, tests show exactly what the code should do.

* *Confidence in changes:* When you optimize a function or fix a bug, tests verify that existing behavior is preserved. This is called regression testing.

* *Edge cases:* Tests force you to think about unusual inputs: empty arrays, single elements, negative numbers, or values at the boundaries of valid ranges.

### Types of Testing

There are several levels of testing, each serving a different purpose:

**Unit tests** verify that individual functions or methods work correctly in isolation. These are the most common type of test and the focus of this lecture.

In [ ]:
def test_mean_of_single_value():
    assert calculate_mean([5.0]) == 5.0

**Integration tests** verify that multiple components work together correctly. For example, testing that a data loading function produces output that a model fitting function can accept.

In [ ]:
def test_load_and_fit_pipeline():
    data = load_dataset("sample.csv")
    model = fit_linear_model(data["X"], data["y"])
    assert model.coef_ is not None

**Regression tests** verify that previously fixed bugs don't reappear. These are often written after discovering a bug.

In [ ]:
def test_variance_with_single_element():
    # Bug #42: variance() crashed on single-element arrays
    result = variance([1.0])
    assert result == 0.0

## Writing Your First Test

Before using any testing framework, let's understand how testing works at its core. Python's built-in `assert` statement is all you need to write basic tests.

### The assert Statement

The `assert` statement checks whether a condition is true. If the condition is false, Python raises an `AssertionError`:

In [ ]:
# This passes silently - no output means success
assert 2 + 2 == 4
print("Assertion passed!")

If we try an assertion that fails, Python raises an `AssertionError`:

```python
assert 2 + 2 == 5  # This would raise AssertionError
```

Output:
```
AssertionError
```

You can add a message to make failures more informative:

```python
assert 2 + 2 == 5, "Expected 5 but got 4"
```

Output:
```
AssertionError: Expected 5 but got 4
```

### A Simple Test

Here's a complete example. Define a function, write a test for it, and run the test:

In [ ]:
def calculate_mean(numbers):
    return sum(numbers) / len(numbers)

def test_mean():
    result = calculate_mean([1, 2, 3, 4, 5])
    assert result == 3.0, f"Expected 3.0, got {result}"

test_mean()
print("test_mean passed")

Before we continue, let's define some helper functions that we'll use in examples throughout this notebook:

Save this as `test_mean.py` and run it:
```
python test_mean.py
```

If the test passes, you'll see:

```
test_mean passed
```

If the test fails (say, if `calculate_mean` had a bug), you'd see an `AssertionError` with your message.

In [ ]:
import numpy as np

def calculate_variance(data):
    """Calculate population variance."""
    data = np.array(data)
    mean = np.mean(data)
    return np.mean((data - mean) ** 2)

def calculate_median(data):
    """Calculate median of data."""
    sorted_data = sorted(data)
    n = len(sorted_data)
    if n % 2 == 1:
        return sorted_data[n // 2]
    return (sorted_data[n // 2 - 1] + sorted_data[n // 2]) / 2

def calculate_std(data):
    """Calculate population standard deviation."""
    return np.sqrt(calculate_variance(data))

def normalize(data):
    """Normalize data to [0, 1] range."""
    data = np.array(data, dtype=float)
    min_val, max_val = np.min(data), np.max(data)
    if max_val == min_val:
        return np.zeros_like(data)
    return (data - min_val) / (max_val - min_val)

def compute_covariance(data):
    """Compute covariance matrix."""
    return np.cov(data, rowvar=False)

def transform(X):
    """Simple identity transform for demonstration."""
    return X.copy()

def remove_missing(data):
    """Remove None values from a list."""
    return [x for x in data if x is not None]

def compute_stats(data):
    """Compute mean and std of data."""
    return {"mean": np.mean(data), "std": np.std(data)}

def bootstrap_mean(data, n_iterations=1000):
    """Compute bootstrap estimate of the mean."""
    means = []
    n = len(data)
    for _ in range(n_iterations):
        sample = data[np.random.randint(0, n, size=n)]
        means.append(np.mean(sample))
    return np.mean(means)

def bootstrap_sample(data, rng=None):
    """Draw a bootstrap sample from data.

    Args:
        data: Input array.
        rng: Random state for reproducibility. If None, uses global state.
    """
    if rng is None:
        rng = np.random.RandomState()
    indices = rng.randint(0, len(data), size=len(data))
    return data[indices]

print("Helper functions defined!")

### Multiple Tests

You can write multiple test functions and call them one by one:

In [ ]:
def add(a, b):
    return a + b

def test_add_positive_numbers():
    assert add(2, 3) == 5

def test_add_negative_numbers():
    assert add(-1, -1) == -2

def test_add_mixed_numbers():
    assert add(-1, 5) == 4

# Run all tests
test_add_positive_numbers()
test_add_negative_numbers()
test_add_mixed_numbers()
print("All tests passed")

### What Makes a Good Test

A good test has a clear purpose and follows a simple structure:

1. Set up the input data
2. Call the function being tested
3. Check that the result matches expectations

Each test should focus on one specific behavior. Instead of testing everything in one function, write separate tests for different cases.

## The AAA Pattern

Well-structured tests follow the AAA pattern: Arrange, Act, Assert. This pattern makes tests readable and maintainable.

### Arrange-Act-Assert Structure

* **Arrange:** Set up the test data and any required state
* **Act:** Execute the code being tested
* **Assert:** Verify the result is correct

In [ ]:
def test_calculate_mean():
    # Arrange
    data = [1.0, 2.0, 3.0, 4.0, 5.0]

    # Act
    result = calculate_mean(data)

    # Assert
    assert result == 3.0

This structure clearly separates what you're testing from how you're testing it. When a test fails, you can quickly identify which phase caused the problem.

### Writing Focused Tests

Each test should verify one specific behavior. Resist the temptation to test multiple things at once.

**Poor structure (multiple behaviors in one test):**

In [ ]:
def test_statistics():
    data = [1, 2, 3, 4, 5]
    assert calculate_mean(data) == 3.0
    assert calculate_median(data) == 3.0
    assert calculate_std(data) > 0
    assert calculate_variance(data) > 0

If this test fails, which function is broken? You'd need to examine the error message carefully to find out.

**Better structure (separate tests):**

In [ ]:
def test_mean():
    data = [1, 2, 3, 4, 5]
    assert calculate_mean(data) == 3.0

def test_median():
    data = [1, 2, 3, 4, 5]
    assert calculate_median(data) == 3.0

def test_std_is_positive():
    data = [1, 2, 3, 4, 5]
    assert calculate_std(data) > 0

Each test has a single purpose and a descriptive name. When one fails, you immediately know which function has a problem.

### Meaningful Test Names

Test names should describe what behavior is being tested. A good pattern is `test_<function>_<scenario>_<expected_result>`:

In [ ]:
def test_mean_with_single_element_returns_that_element():
    assert calculate_mean([42.0]) == 42.0

def test_mean_with_negative_values_handles_correctly():
    assert calculate_mean([-1.0, -2.0, -3.0]) == -2.0

def test_normalize_with_constant_array_returns_zeros():
    result = normalize([5.0, 5.0, 5.0])
    assert all(x == 0.0 for x in result)

These names serve as documentation. Reading the test names gives you a quick summary of what your code does.

### One Assertion Per Test

While not a strict rule, limiting to one logical assertion per test improves clarity. If you need multiple `assert` statements, they should all verify aspects of the same result.

**Acceptable (checking multiple aspects of one result):**

In [ ]:
def test_fit_model_returns_correct_shape():
    X = np.array([[1, 2], [3, 4], [5, 6]])
    y = np.array([1, 2, 3])
    model = fit_model(X, y)

    assert model.coef_.shape == (2,)
    assert isinstance(model.intercept_, float)

**Not ideal (checking unrelated behaviors):**

In [ ]:
def test_model_functionality():
    X = np.array([[1, 2], [3, 4]])
    y = np.array([1, 2])
    model = fit_model(X, y)
    predictions = model.predict(X)

    assert model.coef_ is not None  # Checking fit
    assert len(predictions) == 2    # Checking predict

The second example tests both `fit` and `predict` behavior. These should be separate tests.

### Question

The following test function violates several best practices. Identify the problems and describe how you would refactor it.

In [ ]:
def test_data_processing():
    raw = [1, 2, None, 4, 5]

    cleaned = remove_missing(raw)
    assert len(cleaned) == 4

    normalized = normalize(cleaned)
    assert min(normalized) == 0.0
    assert max(normalized) == 1.0

    stats = compute_stats(normalized)
    assert "mean" in stats
    assert "std" in stats
    assert stats["mean"] > 0

**Answer:**

This test has several problems:

1. **Tests multiple functions in one test.** It tests `remove_missing`, `normalize`, and `compute_stats` together. If any function breaks, the test name doesn't indicate which one.

2. **Each function's output depends on the previous function.** If `remove_missing` has a bug, the `normalize` assertions will also fail, even if `normalize` is correct.

3. **The test name is too vague.** "test_data_processing" doesn't describe what specific behavior is being verified.

Refactored version:

In [ ]:
def test_remove_missing_excludes_none_values():
    raw = [1, 2, None, 4, 5]
    cleaned = remove_missing(raw)
    assert len(cleaned) == 4
    assert None not in cleaned

def test_normalize_scales_to_zero_one_range():
    data = [1.0, 2.0, 4.0, 5.0]
    normalized = normalize(data)
    assert min(normalized) == 0.0
    assert max(normalized) == 1.0

def test_compute_stats_returns_mean_and_std():
    data = [0.0, 0.25, 0.5, 1.0]
    stats = compute_stats(data)
    assert "mean" in stats
    assert "std" in stats

## Testing Numerical Code

Scientific computing introduces unique testing challenges. Floating-point arithmetic doesn't always produce exact results, and numerical algorithms often involve approximations.

### The Floating-Point Problem

Consider this simple arithmetic:

In [ ]:
0.1 + 0.2  # this will give 0.30000000000000004

This is not a Python bug. Binary floating-point cannot represent 0.1 exactly, just as decimal cannot represent 1/3 exactly. These small representation errors accumulate.

This means a naive test will fail:

In [ ]:
def test_sum_no_tolerance():
    result = 0.1 + 0.2
    assert result == 0.3  # This will fail!

# Uncomment to see the failure:
# test_sum_no_tolerance()

### Comparing with Tolerance

To compare floating-point values, check if their difference is smaller than some tolerance:

In [ ]:
def test_sum_with_tolerance():
    result = 0.1 + 0.2
    expected = 0.3
    assert abs(result - expected) < 1e-9

test_sum_with_tolerance()
print("test_sum_with_tolerance passed!")

This checks that `result` is within `1e-9` of `expected`.

### Using math.isclose

Python's standard library provides `math.isclose()` for cleaner tolerance comparisons:

In [ ]:
import math

def test_sum_with_isclose():
    result = 0.1 + 0.2
    assert math.isclose(result, 0.3)

test_sum_with_isclose()
print("test_sum_with_isclose passed!")

By default, `math.isclose()` uses a relative tolerance of `1e-9`, meaning values are considered equal if they differ by less than one part per billion relative to the larger value.

### Controlling Tolerance

Both approaches allow you to specify tolerance explicitly:

In [ ]:
import math

# Absolute tolerance: values within 0.001 of each other
result = 1.001
assert abs(result - 1.0) < 0.01  # passes

# Using math.isclose with explicit tolerances
assert math.isclose(1.0, 1.009, rel_tol=0.01)  # 1% relative tolerance
assert math.isclose(0.001, 0.0019, abs_tol=0.001)  # absolute tolerance

print("All tolerance examples passed!")

**When to use relative vs. absolute tolerance:**

* **Relative tolerance** works well for values of similar magnitude. It scales with the values being compared.
* **Absolute tolerance** is necessary when comparing values near zero, where relative tolerance becomes meaningless.

In [ ]:
def test_near_zero_values():
    result = 1e-10 - 1e-10
    # Use absolute tolerance for values near zero
    assert abs(result - 0.0) < 1e-9

test_near_zero_values()
print("test_near_zero_values passed!")

### Testing NumPy Arrays

For NumPy arrays, you can check element-wise with a loop, but NumPy provides dedicated testing utilities in `np.testing`:

In [ ]:
import numpy as np

def test_array_with_numpy():
    result = np.array([1.0, 2.0, 3.0])
    expected = np.array([1.0, 2.0, 3.0000001])

    # Use np.allclose for comparing arrays with tolerance
    assert np.allclose(result, expected)

test_array_with_numpy()
print("test_array_with_numpy passed!")

NumPy provides convenient functions for comparing arrays:

* `np.allclose(a, b)` - check if all elements are close within tolerance
* `np.isclose(a, b)` - element-wise comparison returning boolean array

### Comparing Array Properties

Sometimes you care about properties of arrays rather than exact values:

In [ ]:
import math

def test_normalize_produces_unit_norm():
    v = np.array([3.0, 4.0])
    result = v / np.linalg.norm(v)  # Normalize to unit vector

    # Check the norm is 1.0 (within tolerance)
    assert math.isclose(np.linalg.norm(result), 1.0)

    # Check the direction is preserved
    assert math.isclose(result[0] / result[1], v[0] / v[1])

def test_covariance_matrix_is_symmetric():
    np.random.seed(42)
    data = np.random.randn(100, 5)
    cov = compute_covariance(data)

    assert np.allclose(cov, cov.T)

def test_output_shape_matches_input():
    np.random.seed(42)
    X = np.random.randn(100, 10)
    result = transform(X)

    assert result.shape == X.shape

# Run all tests
test_normalize_produces_unit_norm()
test_covariance_matrix_is_symmetric()
test_output_shape_matches_input()
print("All tests passed!")

## Reproducibility in Scientific Testing

Scientific code often involves randomness: Monte Carlo simulations, random initializations, bootstrap sampling. Testing such code requires careful handling to get reproducible results.

### Setting Random Seeds

NumPy's random functions draw from a global random state. Setting a seed ensures the same sequence of random numbers:

In [ ]:
import numpy as np

np.random.seed(42)
print(np.random.randn(3))  # Always produces the same values

In tests, set the seed at the beginning:

In [ ]:
import math

def test_bootstrap_mean():
    np.random.seed(12345)
    data = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
    result = bootstrap_mean(data, n_iterations=1000)

    # With seed fixed, we know the expected result
    assert math.isclose(result, 3.0, rel_tol=0.05)

test_bootstrap_mean()
print("test_bootstrap_mean passed!")

### Using RandomState for Isolation

The global `np.random.seed()` affects all random calls, which can cause problems if functions internally use randomness. For better isolation, pass a `RandomState` object to functions that need randomness.

Look back at the `bootstrap_sample` function we defined in the helper functions cell. It accepts an optional `rng` parameter that lets callers control the random state:

```python
def bootstrap_sample(data, rng=None):
    """Draw a bootstrap sample from data.

    Args:
        data: Input array.
        rng: Random state for reproducibility. If None, uses global state.
    """
    if rng is None:
        rng = np.random.RandomState()
    indices = rng.randint(0, len(data), size=len(data))
    return data[indices]
```

In tests, pass an explicit random state:

In [ ]:
def test_bootstrap_sample_reproducibility():
    data = np.array([1.0, 2.0, 3.0, 4.0, 5.0])

    rng1 = np.random.RandomState(42)
    rng2 = np.random.RandomState(42)

    sample1 = bootstrap_sample(data, rng=rng1)
    sample2 = bootstrap_sample(data, rng=rng2)

    assert np.allclose(sample1, sample2)

test_bootstrap_sample_reproducibility()
print("test_bootstrap_sample_reproducibility passed!")

### Testing Statistical Properties

For stochastic algorithms, test statistical properties rather than exact values:

In [ ]:
def generate_standard_normal(n, seed=None):
    """Generate n samples approximating standard normal distribution.
    
    Uses Box-Muller transform to convert uniform random numbers
    to normally distributed values.
    """
    if seed is not None:
        np.random.seed(seed)
    u1 = np.random.random(n)
    u2 = np.random.random(n)
    return np.sqrt(-2 * np.log(u1)) * np.cos(2 * np.pi * u2)

def test_generated_samples_have_correct_mean():
    samples = generate_standard_normal(10000, seed=42)
    # Mean should be close to 0 for large samples
    assert abs(np.mean(samples) - 0.0) < 0.05

def test_generated_samples_have_correct_std():
    samples = generate_standard_normal(10000, seed=42)
    # Std should be close to 1 for large samples
    assert abs(np.std(samples) - 1.0) < 0.05

# Run tests
test_generated_samples_have_correct_mean()
test_generated_samples_have_correct_std()
print("All tests passed!")

The tolerances must account for expected statistical variation.

### Testing Edge Cases

Edge cases are critical in numerical code. Test boundaries and unusual inputs:

In [ ]:
def test_mean_single_element():
    result = calculate_mean(np.array([42.0]))
    assert result == 42.0

def test_variance_constant_array():
    result = calculate_variance(np.array([5.0, 5.0, 5.0, 5.0]))
    assert result == 0.0

def test_normalize_handles_large_values():
    data = np.array([1e10, 2e10, 3e10])
    result = normalize(data)
    assert np.isfinite(result).all()

# Run tests
test_mean_single_element()
test_variance_constant_array()
test_normalize_handles_large_values()
print("All edge case tests passed!")

Testing for expected errors (like checking that a function raises an exception for invalid input) is also possible but requires more advanced Python concepts. For now, focus on testing that your functions return correct results for valid inputs.

## Scaling Up with pytest

The manual testing approach shown above works well for learning and small projects. As your test suite grows, you'll encounter challenges:

* Manually calling each test function becomes tedious
* When a test fails, you have to re-run the entire file
* There's no summary of which tests passed or failed

**pytest** is a testing framework that solves these problems. It automatically discovers and runs tests, provides detailed failure reports, and offers advanced features.

### Installing and Using pytest

Install pytest with pip:
```bash
pip install pytest
```

pytest tests look almost identical to manual tests - the main difference is you don't call the test functions yourself:

```python
# test_math_utils.py
def add(a, b):
    return a + b

def test_add_positive_numbers():
    assert add(2, 3) == 5

def test_add_negative_numbers():
    assert add(-1, -1) == -2

# No need to call test functions - pytest does it automatically
```

Run pytest from the command line:
```bash
pytest              # Run all tests
pytest -v           # Verbose output showing each test
pytest test_file.py # Run tests in specific file
```

pytest automatically discovers all files named `test_*.py` and runs all functions named `test_*`.

For more details on pytest's features like fixtures and parameterization, see the [pytest documentation](https://docs.pytest.org/).

## Best Practices Summary

### Test Isolation and Independence

Each test should be independent. Tests should not:
* Depend on the order they run in
* Share state with other tests
* Require other tests to have run first
* Leave side effects that affect other tests

In [ ]:
# Bad: Tests share state
counter = 0

def test_increment():
    global counter
    counter += 1
    assert counter == 1

def test_counter_value():
    assert counter == 1  # Depends on test_increment running first!

### Test Coverage: Quality Over Quantity

Test coverage measures what percentage of your code is executed by tests. While high coverage is good, it's not sufficient:

In [ ]:
def divide(a, b):
    return a / b

def test_divide():
    assert divide(10, 2) == 5  # 100% coverage, but misses edge case!

This test achieves 100% code coverage but doesn't test division by zero. Focus on testing:
* Normal cases (typical inputs)
* Edge cases (empty inputs, single elements, boundaries)
* Error cases (invalid inputs, exceptions)
* Corner cases specific to your domain (numerical stability, overflow)

### When to Write Tests

There are different philosophies about when to write tests:

**Test-Driven Development (TDD):** Write tests before code. This forces you to think about the interface before implementation.

**Test-After:** Write code first, then tests. This is more common in exploratory scientific work.

**Test-During:** Write tests alongside code, alternating between implementation and testing.

For scientific computing, a practical approach is:
1. Prototype without tests for exploration
2. Add tests once the interface stabilizes
3. Always add a test when you find a bug (regression test)

### Continuous Integration

Tests are most valuable when run automatically. Continuous Integration (CI) services run your tests whenever you push code changes. Popular options include GitHub Actions, GitLab CI, and Travis CI.

A typical CI workflow:
1. Developer pushes code to repository
2. CI server detects the push
3. CI runs all tests
4. Developer is notified if any tests fail

This catches bugs early and ensures tests are always run, not just when developers remember to run them locally.

### Test File Organization

For larger projects, keep test files separate from your source code:

```
myproject/
    mypackage/
        stats.py
        models.py
    tests/
        test_stats.py
        test_models.py
```

## Recommended Resources

* [pytest documentation](https://docs.pytest.org/) - Official pytest documentation
* [Real Python - Effective Python Testing With pytest](https://realpython.com/pytest-python-testing/) - Comprehensive pytest tutorial
* [Scientific Python Development Guide - Testing](https://learn.scientific-python.org/development/tutorials/test/) - Testing practices for scientific Python
* [NumPy Testing Guidelines](https://numpy.org/doc/stable/reference/routines.testing.html) - NumPy's testing utilities